# Exploring Topic and Word Relationships

Here we explore different ways that the the newstest 2015 dataset can be looked at.

In [2]:
import math

def read_file(file_path, max_amount=math.inf):
    """
    Reads a number of lines from file and return list of lines
    :param file_path:
    :param max_amount:
    :return: list[str]
    """
    line_list = []

    with open(file=file_path) as f:
        curr_idx = 0
        while curr_idx < max_amount:
            line_list.append(f.readline())
            curr_idx += 1

    return line_list

print("Loading file")
l_raw = read_file('./trg.shuf', 1000000)
print("Finished loading in file")

Loading file
Finished loading in file


In [10]:
import math
import pandas as pd
from collections import Counter
import operator
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
import gensim
from gensim import corpora





def process_word(word):
    """
    Returns process word
    :param word:
    :return: str word
    """
    # lower
    word = word.lower()
    
    # Remove stop
    if word in en_stop:
        return None

    # Remove symbols
    if len(word) < 3:
        return None
    
    # Get morph
    t_word = wn.morphy(word)
    if t_word is not None:
        word = t_word

    # Get lemma
    word = WordNetLemmatizer().lemmatize(word)
    
    return word


def process_bpe(line_list, seq="@@", rm_words=["\n"]):
    """
    Transforms line list form read_file to list[list[str=words]], and bpe words merged together
    :param line_list:
    :param seq:
    :param rm_words:
    :return: ist[list[str=words]]
    """
    full_word_list = []
    for line in line_list:
        temp_words = line.split()
        full_words = []
        idx = 0
        while idx < len(temp_words):
            if temp_words[idx].endswith(seq):
                temp_str = ""
                while temp_words[idx].endswith(seq):
                    temp_str += temp_words[idx][:-(len(seq))]
                    idx += 1
                temp_str += temp_words[idx]
                if temp_str not in rm_words:
                    w = process_word(temp_str)
                    if w is not None:
                        full_words.append(w)
                idx += 1
            else:
                if temp_words[idx] not in rm_words:
                    w = process_word(temp_words[idx])
                    if w is not None:
                        full_words.append(w)
                idx += 1
        full_word_list.append(full_words)
    return full_word_list


def get_word_count(word_list):
    """
    Returns word count.
    :param word_list:
    :return: List[(str=word, int=count)]
    """
    word_count = {}
    for line in word_list:
        for word, val in Counter(line).most_common():
            if word in word_count:
                word_count[word] += val
            else:
                word_count[word] = val

    return sorted(word_count.items(), key=operator.itemgetter(1))


def get_topics_lda(word_list, num_topics=10):
    dictionary = corpora.Dictionary(word_list)
    corpus = [dictionary.doc2bow(text) for text in word_list]

    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

    return topics, ldamodel, corpus, dictionary

import random
l = random.sample(l_raw, 100000)

wl = process_bpe(line_list=l)
print(get_word_count(word_list=wl))
t, ldamodel,_, dictionary = get_topics_lda(word_list=wl)




[nltk_data] Downloading package wordnet to /home/nikita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nikita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('flegt', 1), ('185.58', 1), ('eco-innovations', 1), ('super-credits', 1), ('2366', 1), ('pnnl', 1), ('self-adhesive', 1), ('insultation', 1), ('lifelike', 1), ('westenhofer', 1), ('breathalyse', 1), ('ostrowski', 1), ('988', 1), ('3603868', 1), ('12.10.2009', 1), ('juiced', 1), ('adrenaline-charged', 1), ('53095', 1), ('4367514', 1), ('eib-universities', 1), ('re-launch', 1), ('201310311537', 1), ('textrace-textile-rfid-loesung-praesentiert-im-tyco-retail-experience-center.html', 1), ('bye', 1), ('mdi', 1), ('18ht', 1), ('www.katatonia.com', 1), ('22.5', 1), ('festspielhaus', 1), ('00-49-', 1), ('8821-750-2469', 1), ('stelios', 1), ('christen', 1), ('al-zamili', 1), ('morton', 1), ('paige', 1), ('0650613-519', 1), ('1918-1925', 1), ('prod.nr.', 1), ('21868', 1), ('wunderhorn', 1), ('pentahotels', 1), ('weiden', 1), ('12.01.1990', 1), ('anti-french', 1), ('fh0244.html', 1), ('08029', 1), ('mankiw', 1), ('2141', 1), ('enjoyyour', 1), ('sahrawi', 1), ('20090348.htm', 1), ('27.06.2015', 

(0, '0.025*"year" + 0.018*"say" + 0.016*"new" + 0.013*"last" + 0.009*"second" + 0.008*"week" + 0.008*"page" + 0.008*"first" + 0.006*"day" + 0.005*"two"')
(1, '0.016*"hotel" + 0.012*"city" + 0.011*"room" + 0.010*"offer" + 0.008*"visit" + 0.008*"day" + 0.007*"number" + 0.007*"area" + 0.006*"centre" + 0.006*"stay"')
(2, '0.017*"..." + 0.009*"case" + 0.009*"please" + 0.008*"time" + 0.007*"god" + 0.007*"contact" + 0.005*"version" + 0.005*"aid" + 0.005*"love" + 0.005*"follow"')
(3, '0.008*"information" + 0.006*"price" + 0.006*"person" + 0.006*"use" + 0.006*"data" + 0.006*"website" + 0.005*"user" + 0.005*"especially" + 0.005*"tel" + 0.005*"per"')
(4, '0.011*"one" + 0.011*"make" + 0.010*"would" + 0.009*"also" + 0.009*"say" + 0.008*"time" + 0.008*"take" + 0.008*"way" + 0.008*"like" + 0.007*"people"')
(5, '0.009*"house" + 0.007*"going" + 0.006*"guest" + 0.006*"wish" + 0.006*"online" + 0.006*"found" + 0.005*"release" + 0.005*"room" + 0.005*"image" + 0.005*"stand"')
(6, '0.040*"night" + 0.026*"tim




# Visualization
Here we visualize with pyLDAvis in an interactive mode the LDA analysis.

In [12]:
import pyLDAvis.gensim
from gensim import corpora
dictionary = corpora.Dictionary(wl)
corpus = [dictionary.doc2bow(text) for text in wl]
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [16]:
def get_representatives_lda(word_list, lda_model, dictionary, num_topics=10):
    max_line = [[]] * num_topics
    max_class = [0] * num_topics 
    class_vocab = {i:{} for i in range(0, num_topics)}
    
    for line in word_list:
        new_doc_bow = dictionary.doc2bow(line)
        s = ldamodel.get_document_topics(new_doc_bow)
        s.sort(key=lambda x: x[1])
        t_max_class, t_max_perc = s[-1]
        
        for word in line:
            if word in class_vocab[t_max_class]:
                class_vocab[t_max_class][word] += 1
            else:
                class_vocab[t_max_class][word] = 1
        
        if max_class[t_max_class] < t_max_perc:
            max_class[t_max_class] = t_max_perc
            max_line[t_max_class] = line
    
    return max_line, max_class, class_vocab

max_l, max_c, class_vocab = get_representatives_lda(wl, ldamodel, dictionary)
        

"""
for idx in range(len(max_l)):
    print("\n")
    print("\n")
    print(max_c[idx])
    print(max_l[idx])
"""    

'\nfor idx in range(len(max_l)):\n    print("\n")\n    print("\n")\n    print(max_c[idx])\n    print(max_l[idx])\n'

In [19]:
# get overlap
num_topics = 10

import numpy as np


def get_overlap_matrix(class_vocab, num_topics):
    over_lap_matrix = np.ones((num_topics, num_topics))

    for c in range(0, num_topics):
        for c2 in range(0, num_topics):
            # Skip self
            if c2 == c:
                continue
            # overlap
            words_in_c2_and_c1 = 0
            total_words_c2 = 0
            for word in class_vocab[c2]:
                total_words_c2 += 1  # class_vocab[c2][word]
                if word in class_vocab[c]:
                    words_in_c2_and_c1 += 1
            over_lap_matrix[c, c2] = words_in_c2_and_c1/(len(class_vocab[c2]) + len(class_vocab[c]))
    
    return over_lap_matrix


over_lap_matrix = get_overlap_matrix(class_vocab, num_topics)

print("Overlap Matrix of Vocab")
print(pd.DataFrame(over_lap_matrix))
print("\n")

print("Average for each topic")
# Remove diagonal as it skews data
print(pd.DataFrame(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1), axis=1)))
print("\n")

print("Average overall")
print(np.average(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))

print("Max")
print(np.max(over_lap_matrix[~np.eye(over_lap_matrix.shape[0],dtype=bool)].reshape(over_lap_matrix.shape[0],-1)))
                

Overlap Matrix of Vocab
          0         1         2         3         4         5         6  \
0  1.000000  0.197624  0.200882  0.196976  0.220141  0.201617  0.166450   
1  0.197624  1.000000  0.195837  0.190657  0.201090  0.198069  0.166539   
2  0.200882  0.195837  1.000000  0.198589  0.198970  0.200185  0.172255   
3  0.196976  0.190657  0.198589  1.000000  0.200631  0.191327  0.172573   
4  0.220141  0.201090  0.198970  0.200631  1.000000  0.197456  0.159661   
5  0.201617  0.198069  0.200185  0.191327  0.197456  1.000000  0.170049   
6  0.166450  0.166539  0.172255  0.172573  0.159661  0.170049  1.000000   
7  0.122414  0.122767  0.127916  0.123171  0.110115  0.124541  0.121561   
8  0.132579  0.130084  0.134319  0.130052  0.126426  0.131855  0.122310   
9  0.206708  0.191883  0.195866  0.196134  0.242586  0.192082  0.162804   

          7         8         9  
0  0.122414  0.132579  0.206708  
1  0.122767  0.130084  0.191883  
2  0.127916  0.134319  0.195866  
3  0.123171  0